In [157]:
import numpy as np
from time import process_time
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

Question_1

In [ ]:
WORLD_SIZE = (8, 8)
START_POSITION = [3, 1]
OBSTACLES_POSITIONS = [[0, 6], [1, 6], [2, 6], [2, 2], [3, 2], [4, 2], [6, 4]]
GOAL_POSITION = [0, 7]
"""
Action mapping:
    0: Left
    1: Down
    2: Right
    3: Up
"""
ACTIONS = [0, 1, 2, 3]

ACTION_PROB = 0.25

In [142]:
def one_step(state, action):

    if state == GOAL_POSITION:
        return  START_POSITION, 100
    # if agent hits to an obstacle then it stays on previous state
    if state in OBSTACLES_POSITIONS:
        return state, 0

    if action==0: # left
        next_state = np.array(state) + np.array([0, -1])
    if action==1: # down
        next_state = np.array(state) + np.array([1, 0])
    if action==2: # right
        next_state = np.array(state) + np.array([0, 1])
    if action==3: # up
        next_state = np.array(state) + np.array([-1, 0])
    
    x, y = next_state
    # check if agent goes out.
    if x < 0 or x >= WORLD_SIZE[0] or y < 0 or y >= WORLD_SIZE[1]:
        reward = 0
        # stay on previous state
        next_state = state
    else:
        reward = 0
    return next_state, reward

In [143]:
def policy_evaluation(discount_factor=0.9, tol=1e-5):
    
    state_values = np.zeros(WORLD_SIZE, dtype=np.float)
    while True:
        delta = 0
        # Iterate over all states
        new_value = np.zeros_like(state_values)
        for i in range(WORLD_SIZE[0]):
            for j in range(WORLD_SIZE[1]):
                current_value = state_values[i][j]
                for action in ACTIONS:
                    (next_i, next_j), reward = one_step([i, j], action)

                    new_value[i][j] +=ACTION_PROB * (reward + (discount_factor * state_values[next_i, next_j]))

                delta = max(delta, np.absolute(new_value[i][j] - current_value))   
        if tol > delta:
            break
        state_values = new_value
        
    return state_values

In [154]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.round(policy_evaluation(),decimals=3)]))

0.002	0.003	0.006	0.012	0.017	0.015	0.0	100.001
0.002	0.003	0.006	0.019	0.032	0.035	0.0	32.052
0.001	0.001	0.0	0.032	0.072	0.11	0.0	10.402
0.001	0.001	0.0	0.051	0.145	0.381	1.106	3.777
0.001	0.001	0.0	0.051	0.139	0.335	0.755	1.503
0.003	0.004	0.011	0.037	0.088	0.214	0.412	0.644
0.003	0.005	0.008	0.013	0.0	0.116	0.218	0.303
0.003	0.004	0.008	0.014	0.028	0.082	0.14	0.181


Question_2

In [130]:
def print_policy(policy):
        
    actions = []
    for (i,j), val in np.ndenumerate(policy):
            
            if [i,j] in OBSTACLES_POSITIONS:
                actions.append("#")
            elif [i,j] == GOAL_POSITION:
                actions.append("#")
            elif val == 0:
                actions.append("←")
            elif val == 1:
                actions.append("↓")
            elif val == 2:
                actions.append("→")
            elif val == 3:
                actions.append("↑")
                
    return np.reshape(actions, newshape=(8,8))

In [175]:
def policy_evaluation_version_2(policy, tol=1e-5, discount_factor=0.9):
    
    state_values = np.zeros(WORLD_SIZE, dtype=np.float)
    while True:
        delta = 0
        # Iterate over all states
        for i in range(WORLD_SIZE[0]):
            for j in range(WORLD_SIZE[1]):
                current_value = state_values[i, j]

                (next_i, next_j), reward = one_step([i, j], policy[i][j])
                
                state_values[i, j] = 1 * (reward + (discount_factor * state_values[next_i, next_j]))
                
                delta = max(delta, np.absolute(current_value - state_values[i, j]))
                
        if tol > delta:
            break
        
    return np.around(state_values, decimals=3)

def policy_improvement(value_from_policy, discount_factor=0.9):

    new_policy = np.zeros(WORLD_SIZE, dtype=np.int)

    for i in range(WORLD_SIZE[0]):
        for j in range(WORLD_SIZE[1]):
            
            action_value = np.zeros(len(ACTIONS), dtype=np.int)
            for action in ACTIONS:
                val_s_a = 0
                (next_i, next_j), reward = one_step([i, j], action)
                
                val_s_a =  1 * \
                    (reward + (discount_factor * value_from_policy[next_i, next_j]))
            
                action_value[action] = val_s_a
            
            new_policy[i][j] = np.argmax(action_value)
                    
    return new_policy

def policy_iteration():
    
    value_function = np.zeros(WORLD_SIZE, dtype=np.float)
    states_policy = np.zeros(WORLD_SIZE, dtype=np.int)
    
    while True:
        
        temp_val_func = policy_evaluation_version_2(states_policy)
        improved_policy = policy_improvement(temp_val_func)
    
        if np.array_equal(states_policy, improved_policy):
            break
        else:
           states_policy = improved_policy

    value_function = temp_val_func
    
    return value_function, states_policy

value, action = policy_iteration()

print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in print_policy(action)]))
print("\n\n")
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in value]))

↓	↓	↓	↓	↓	↓	#	#
→	→	→	↓	↓	↓	#	↑
→	↑	#	↓	↓	↓	#	↑
↓	↓	#	→	→	→	→	↑
↓	↓	#	→	→	→	→	↑
→	→	→	→	→	→	→	↑
→	→	→	↑	#	→	→	↑
→	→	→	→	→	→	→	↑



32.959	36.621	40.69	45.211	50.234	55.816	0.0	129.663
36.621	40.69	45.211	50.234	55.816	62.017	0.0	116.696
32.959	36.621	0.0	55.816	62.017	68.908	0.0	105.027
29.663	32.959	0.0	62.017	68.908	76.564	85.072	94.524
32.959	36.621	0.0	55.816	62.017	68.908	76.565	85.072
36.621	40.69	45.211	50.234	55.816	62.017	68.908	76.565
32.959	36.621	40.69	45.211	0.0	55.816	62.017	68.908
29.663	32.959	36.621	40.69	45.211	50.234	55.816	62.017


Question_3

In [176]:
def value_iteration(discount_factor=0.9, tol=1e-8):
    
    value_function = np.zeros(WORLD_SIZE, dtype=np.float)
    states_policy = np.zeros(WORLD_SIZE, dtype=np.int)
    
    while True:
        delta = 0
        for i in range(WORLD_SIZE[0]):
            for j in range(WORLD_SIZE[1]):
                current_value = value_function[i, j]
                
                val = np.zeros(len(ACTIONS))
                for action in ACTIONS:
                    (next_i, next_j), reward = one_step([i, j], action)
                
                    val[action] =  1 * \
                        (reward + (discount_factor * value_function[next_i, next_j]))

                states_policy[i][j] = np.argmax(val)
                value_function[i][j] = np.max(val)
                delta = max(delta, np.absolute(current_value - value_function[i, j])) 

        if tol > delta:
            break
    
    return value_function, states_policy

value, action = value_iteration()

print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in print_policy(action)]))
print("\n\n")
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.round(value, decimals=3)]))

↓	↓	↓	↓	↓	↓	#	#
→	→	→	↓	↓	↓	#	↑
→	↑	#	↓	↓	↓	#	↑
↓	↓	#	→	→	→	→	↑
↓	↓	#	→	→	→	→	↑
→	→	→	→	→	→	→	↑
→	→	→	↑	#	→	→	↑
→	→	→	→	→	→	→	↑



32.959	36.621	40.69	45.211	50.234	55.816	0.0	129.663
36.621	40.69	45.211	50.234	55.816	62.017	0.0	116.696
32.959	36.621	0.0	55.816	62.017	68.908	0.0	105.027
29.663	32.959	0.0	62.017	68.908	76.565	85.072	94.524
32.959	36.621	0.0	55.816	62.017	68.908	76.565	85.072
36.621	40.69	45.211	50.234	55.816	62.017	68.908	76.565
32.959	36.621	40.69	45.211	0.0	55.816	62.017	68.908
29.663	32.959	36.621	40.69	45.211	50.234	55.816	62.017


In [185]:
policy_iteration_runtimes = []
for _ in range(1000):
    t1_start = process_time()
    policy_iteration()
    t1_stop = process_time()
    policy_iteration_runtimes.append(t1_stop - t1_start)
    
print("Elapsed time during Policy Iteration in seconds:", np.average(policy_iteration_runtimes))

Elapsed time during Policy Iteration in seconds: 0.2067340630000002


In [186]:
value_iteration_runtimes = []
for _ in range(1000):
    t1_start = process_time()
    value_iteration()
    t1_stop = process_time()
    policy_iteration_runtimes.append(t1_stop - t1_start)

print("Elapsed time during Policy Iteration in seconds:", np.average(policy_iteration_runtimes))

Elapsed time during Policy Iteration in seconds: 0.2913868874999991
